In [1]:
import numpy as np
from sklearn.datasets import load_iris, load_digits,load_wine,load_breast_cancer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import preprocessing

rbf核函数：$K(x, x')=exp(-||x-x'||^2) \over 2\sigma^2$

In [2]:
def kernel_trans(X, x, kernel):
    if kernel[0] == "linear":
        k = np.dot(X, x.T)
        return k
    elif kernel[0] == 'rbf':
        k = np.zeros((X.shape[0], ))
        for i in range(X.shape[0]):
            k[i] = np.dot(X[i, :] - x, X[i, :] - x)
        k = np.exp(k / (-2 * kernel[1] ** 2))
        return k

$f(x) = w^T\phi(x) + b = \sum_{i=1}^{m} {\alpha_iy_i\kappa(x,x_i)+b}$

In [4]:
def f(i, cache):
    return np.dot(cache['K'][i, :], cache['alphas']*cache['y']) + cache['b']

$E_i = f(x_i) - y_i$

In [5]:
def E(i, cache):
    fxi = f(i, cache)
    return fxi - float(cache['y'][i])

根据我们之前推导的式子$\alpha_{2}^{new, unclipped} = \alpha_{2}^{old} + \frac{y_{2}(E_{1} - E_{2})}{\eta}$ 可以知道第二个向量𝑎_𝑗选择使|𝐸_1−𝐸_2 |最大的向量。

In [6]:
def select_j(i, Ei, cache):
    max_j = None
    max_sub = 0
    max_Ej = None
    for j in range(cache['alphas'].shape[0]):
        if i == j:
            continue
        Ej = E(j, cache)
        sub = abs(Ei - Ej)
        if sub > max_sub:
            max_sub = sub
            max_Ej = Ej
            max_j = j
    return max_j, max_Ej

In [7]:
def clip_alpha(alpha, L, H):
    if alpha > H:
        alpha = H
    elif alpha < L:
        alpha = L
    return alpha

## SMO算法
1. 初始化
 选取一对需更新的𝑎_𝑖、𝑎_𝑗
2. 𝑎_𝑗更新与截取：
$$	𝑎_𝑗^{𝑛𝑒𝑤}=𝑎_𝑗^{𝑜𝑙𝑑}+(𝑦_2 (𝐸_1−𝐸_2))/𝐾，其中𝐾=𝐾_{𝑖𝑖}+𝐾_{𝑗𝑗}−2𝐾_{𝑖𝑗}$$
	截取𝑎_𝑗使其满足 $𝑎_𝑗∈[L,H]$  
    如果$y_1,y_2$异号  
    $L=\max(0, a_2-a_1) $   $ H=\min(C, C+a_2-a_1) $  
    如果$y_1,y_2$同号  
    $L=\max(0,a_2+a_1-C) $   $ H=\min(C,a_2+a_1)$
	
4. 𝑎_𝑖 更新：
	$$𝑎_𝑖^{𝑛𝑒𝑤}=𝑎_𝑖^{𝑜𝑙𝑑}+𝑦_1 𝑦_2 (𝑎_𝑗^{𝑜𝑙𝑑}−𝑎_𝑗^{𝑛𝑒𝑤})$$
5. b更新：
	$$𝑏_1^{𝑛𝑒𝑤}=𝑏^{𝑜𝑙𝑑}−𝐸_𝑖−𝑦_𝑖 𝐾_{𝑖𝑖} (𝑎_𝑖^{𝑛𝑒𝑤}−𝑎_𝑖^{𝑜𝑙𝑑} )−𝑦_𝑗 𝐾_{𝑖𝑗} (𝑎_𝑗^{𝑛𝑒𝑤}−𝑎_𝑗^{𝑜𝑙𝑑})$$
	$$𝑏_2^{𝑛𝑒𝑤}=𝑏^{𝑜𝑙𝑑}−𝐸_𝑗−𝑦_𝑖 𝐾_{𝑖𝑗} (𝑎_𝑖^{𝑛𝑒𝑤}−𝑎_𝑖^{𝑜𝑙𝑑} )−𝑦_𝑗 𝐾_{𝑗𝑗} (𝑎_𝑗^{𝑛𝑒𝑤}−𝑎_𝑗^{𝑜𝑙𝑑})$$
	$若0<𝑎_𝑖^{𝑛𝑒𝑤}<𝐶，则𝑏^{𝑛𝑒𝑤}=𝑏_1^{𝑛𝑒𝑤}$  
	$若0<𝑎_𝑗^{𝑛𝑒𝑤}<𝐶，则𝑏^{𝑛𝑒𝑤}=𝑏_2^{𝑛𝑒𝑤}$  
	$否则，𝑏^{𝑛𝑒𝑤}=(𝑏_1^{𝑛𝑒𝑤}+𝑏_2^{𝑛𝑒𝑤})/2$
6. 若达到终止条件停止算法，否则转到2.

In [9]:
def update_alpha(i, cache):
    alphas = cache['alphas']
    X = cache['X']
    y = cache['y']
    Ei = E(i, cache)
    if (y[i] * Ei < -cache['toler'] and alphas[i] < cache['C']) or (y[i] * Ei > cache['toler'] and alphas[i] > 0):
        j, Ej = select_j(i, Ei, cache)
        alpha_i_old = alphas[i].copy()
        alpha_j_old = alphas[j].copy()
        if y[i] != y[j]:
            L = max(0, alphas[j] - alphas[i])
            H = min(cache['C'], cache['C'] + alphas[j] - alphas[i])
        else:
            L = max(0, alphas[j] + alphas[i] - cache['C'])
            H = min(cache['C'], alphas[j] + alphas[i])
        # if L == H:
        #     print("L==H")
        #     return 0
        kii = cache['K'][i][i]
        kjj = cache['K'][j][j]
        kij = cache['K'][i][j]
        eta = kii + kjj - 2.0 * kij
        if eta == 0:
            print("eta==0")
            return 0
        alphas[j] += y[j] * (Ei - Ej) / eta
        alphas[j] = clip_alpha(alphas[j], L, H)
        if abs(alphas[j] - alpha_j_old) < 0.00001:
            # print("j not moving enough")
            return 0
        alphas[i] += y[i] * y[j] * (alpha_j_old - alphas[j])
        b1 = cache['b'] - Ei - y[i] * kii * (alphas[i] - alpha_i_old) - y[j] * kij * (alphas[j] - alpha_j_old)
        b2 = cache['b'] - Ej - y[i] * kij * (alphas[i] - alpha_i_old) - y[j] * kjj * (alphas[j] - alpha_j_old)
        if 0 < alphas[i] < cache['C']:
            cache['b'] = b1
        elif 0 < alphas[j] < cache['C']:
            cache['b'] = b2
        else:
            cache['b'] = (b1 + b2) / 2
        return 1
    else:
        return 0

软间隔
$$ \max_\alpha  {\sum_{i=1}^{m}{\alpha_i} - {1 \over 2}\sum_{i=1}^{m}{\sum_{j=1}^{m}{\alpha_i\alpha_jy_iy_j{x_i}^Tx_j}}} $$
$$ s.t. \sum_{i=1}^{m}{\alpha_iy_i=0}$$
$$ 0 \leq \alpha_i \leq C, i=1,2,....,m$$

In [10]:
class SVM(object):
    def __init__(self, kernel=("linear", )):
        self.alphas = None
        self.b = 0
        self.kernel = kernel

    def fit(self, X, y, toler, C):
        cache = {}
        cache['X'] = X
        cache['y'] = y
        self.y = y
        self.X = X
        m = y.shape[0]
        cache['toler'] = toler
        cache['C'] = C
        cache['alphas'] = np.zeros((m, 1))
        cache['b'] = 0
        cache['K'] = np.zeros((m, m))
        for i in range(m):
            cache['K'][:, i] = kernel_trans(X, X[i, :], self.kernel)
        alpha_pairs_changed = 0
        entire = True
        # 第一个变量的选择为外循环，在整个样本集和非边界样本集间进行交替:
        while alpha_pairs_changed > 0 or entire:
            alpha_pairs_changed = 0
            if entire:
                for i in range(X.shape[0]):
                    alpha_pairs_changed += update_alpha(i, cache)
            else:
                non_bound = [i for i in range(m) if 0 < cache['alphas'][i] < cache['C']]
                for i in non_bound:
                    alpha_pairs_changed += update_alpha(i, cache)
            if entire:
                entire = False
            elif alpha_pairs_changed == 0:
                entire = True
        self.alphas = cache['alphas']
        self.b = cache['b']
        print("支持向量个数：", np.sum(cache['alphas'] != 0))
        # print(cache['alphas'])

    def predict(self, X_test):
        pred = []
        m = X_test.shape[0]
        K = np.zeros((m, self.X.shape[0]))
        for i in range(m):
            K[:, i] = kernel_trans(X_test, self.X[i, :], self.kernel)
        pred = np.dot(K, self.alphas*self.y) + self.b
        pred = np.where(pred > 0, 1, -1)
        return pred

    def evaluate(self, X, y):
        pred = self.predict(X)
        return np.sum(pred == y) / X.shape[0]

In [11]:
def plot(X, y, w, b):
    # 绘图
    fig = plt.figure()
    ax = fig.add_subplot(111)
    class0 = np.where(y == -1)
    class1 = np.where(y == 1)
    ax.scatter(X[class0[0], 0], X[class0[0], 1], label="class0")
    ax.scatter(X[class1[0], 0], X[class1[0], 1], label="class1")

    x = range(int(min(X[:, 0])), int(max(X[:, 0])) + 1)
    y = (-b - w[0] * x) / w[1]
    ax.plot(x, y)
    plt.show()

In [16]:
# X, y = load_iris(return_X_y=True)
# X, y = X[:100, :], y[:100]
X, y = load_breast_cancer(return_X_y=True)
X = preprocessing.scale(X)
print(X.shape, y.shape)
y = np.where(y == 0, -1, 1).reshape(y.shape[0], 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.3)
# X_train, X_test, y_train, y_test = X[0:398, :], X[398:, :], y[0:398], y[398:]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
svm_linear = SVM()
svm_linear.fit(X_train, y_train, 0.001, 1)
pred = svm_linear.predict(X_test)
score = svm_linear.evaluate(X_train, y_train)
print("linear_train_score；", score)
score = svm_linear.evaluate(X_test, y_test)
print("linear_test_score:", score)
svm_rbf = SVM(kernel=('rbf', 0.1))
svm_rbf.fit(X_train, y_train, 0.001, 1)
pred = svm_rbf.predict(X_test)
score = svm_rbf.evaluate(X_train, y_train)
print("rbf_train_score；", score)
score = svm_rbf.evaluate(X_test, y_test)
print("rbf_test_score:", score)
# plot(X_train, y_train, mysvm.w, mysvm.b)
# plot(X_test, y_test, mysvm.w, mysvm.b)

sk_svm = SVC()
sk_svm.fit(X_train, y_train)
sk_svm.predict(X_test)
print("sklearn train:", sk_svm.score(X_train, y_train))
print("sklearn test:", sk_svm.score(X_test, y_test))

(569, 30) (569,)
(398, 30) (398, 1) (171, 30) (171, 1)
支持向量个数： 6
linear_train_score； 0.891959798995
linear_test_score: 0.719298245614
支持向量个数： 286
rbf_train_score； 0.962311557789
rbf_test_score: 0.894736842105
sklearn train: 0.992462311558
sklearn test: 0.947368421053


E:\software\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
